In [7]:
import pandas as pd
import numpy as np
from rapidfuzz import fuzz, process
from rank_bm25 import BM25Okapi
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import os
from time import time
openai_api_key = os.getenv('OPENAI_API_KEY')
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [9]:
excel_file = '../data-dockyard/250820_Indirect spend analysis_v11.xlsx'
xls = pd.ExcelFile(excel_file)
xls.sheet_names

['Index',
 'Savings potential',
 'Category wise spend',
 'AVD_3Y',
 'Single sourcing_1Y',
 'Negotiation potential',
 'Vendors and SW for negotiation',
 'Rate contract check',
 'Tail spend_by SKU',
 'Tail spend_by vendor',
 'Services contract pooling',
 'Global marketing',
 'Marketing_Alternate vendor',
 '1Y data_Suresh',
 'Sheet3',
 'Sheet2',
 'Marketing spend split',
 '3Y data_Raneeth',
 'FY22-23',
 'FY23-24',
 'FY24-25']

In [10]:
df_tailsku = pd.read_excel(excel_file, sheet_name='Tail spend_by SKU', skiprows=11564)

In [11]:
df_tailsku = df_tailsku.drop(columns = ['Unnamed: 0','Unnamed: 5'])

In [13]:
# limit analysis to skus which cover 80% of the spends
df_tailsku['Spend % cum'] = df_tailsku['Spend %'].cumsum()
cutoff = 0.8 
mvp_skus_cutoff = df_tailsku[df_tailsku['Spend % cum'] >= cutoff].index[0]
df_tailsku_mvp = df_tailsku.iloc[:mvp_skus_cutoff, :]

In [14]:
print(f"{round(100*df_tailsku_mvp.shape[0]/df_tailsku.shape[0])}% ({df_tailsku_mvp.shape[0]}) of skus make up {int(100*cutoff)}% of spends!")

5% (573) of skus make up 80% of spends!


In [19]:
# Build universe of searchable SKUs 
SKU_UNIVERSE_LIST = df_tailsku_mvp['Short Text'].values

In [21]:
SKU_UNIVERSE_LIST[:4]

array(['Supply Chain Related Cost', 'Fulphila Copay',
       'BIO-PERTUZ-301- ChO2', 'Distribution & SCM Related'], dtype=object)

----

In [42]:
tic = time()
client = OpenAI()
embeddings = client.embeddings.create(input=SKU_UNIVERSE_LIST, model='text-embedding-3-small').data
toc = time()
print(f"Time taken : {round(toc - tic)}s")

Time taken : 2s


In [46]:
# with open('embeddings_SKU_UNIVERSE_LIST.pkl','wb') as f:
#     pickle.dump(embeddings, f)

In [68]:
def search_skus(user_query, n=3, search_algo='random', sku_universe=SKU_UNIVERSE_LIST, embeddings=None):
    print(f'\nSearching for {user_query} using {search_algo} ...\n')
    search_results = []
    if search_algo == 'random':    
        search_results = np.random.choice(sku_universe, size=n, replace=False)
    elif search_algo == 'fuzzy':
        search_results = process.extract(user_query, sku_universe, scorer=fuzz.token_set_ratio, limit=n)
    elif search_algo == 'bm25':
        tokenized_sku_universe = [doc.split(" ") for doc in sku_universe]
        bm25 = BM25Okapi(tokenized_sku_universe)
        tokenized_user_query = user_query.split(" ")
        search_results = bm25.get_top_n(tokenized_user_query, sku_universe, n=n)
    elif search_algo == 'semantic':
        if embeddings is None:
            raise Exception('Must pass embeddings for semantic search')
        client = OpenAI()
        query_embedding = client.embeddings.create(input=user_query, model='text-embedding-3-small').data[0].embedding
        df_sku_universe = pd.DataFrame({'sku_short_text':sku_universe, 'embeddings':embeddings})
        df_sku_universe['similarity'] = df_sku_universe['embeddings'].apply(lambda x : cosine_similarity(np.array(query_embedding).reshape(1,-1), 
                                                                                                         np.array(x.embedding).reshape(1,-1))[0][0])
        search_results = df_sku_universe.sort_values('similarity', ascending=False).head(n)['sku_short_text'].values
    else:
        raise Exception(f"Unknown search algo! Please choose one of the following : random, fuzzy, bm25, semantic")
    print(f'Finished Searching!')
    for e, r in enumerate(search_results):
        print(e+1,r)
    print('------------------------------------')
    return search_results

In [70]:
N = 3
search_queries = ['TSA', 'TriSoyAgar Plates', 'Bio-Pertuz']
search_algos = ['fuzzy', 'bm25','semantic']

In [74]:
for user_query in search_queries:
    for search_algo in search_algos:
        search_results = search_skus(user_query=user_query, search_algo=search_algo, n=N, embeddings=embeddings)


Searching for TSA using fuzzy ...

Finished Searching!
1 ('TSA 3P IRR Neutralizers', 100.0, 35)
2 ('STORAGE', 40.0, 470)
3 ('AT PHARMA SPA', 37.5, 61)
------------------------------------

Searching for TSA using bm25 ...

Finished Searching!
1 TSA 3P IRR Neutralizers
2 IQVIA - ANZ Consulting (1st Instalment)
3 SAP Inf Coll Hub for LS Base,LSSCPC
------------------------------------

Searching for TSA using semantic ...

Finished Searching!
1 TSA 3P IRR Neutralizers
2 TICEL HVAC
3 TRANSPORATION CHARGES
------------------------------------

Searching for TriSoyAgar Plates using fuzzy ...

Finished Searching!
1 ('professional fees', 41.1764705882353, 11)
2 ('TSA 3P IRR Neutralizers', 40.0, 35)
3 ('Pegasus Project Charges_ JSA', 40.0, 474)
------------------------------------

Searching for TriSoyAgar Plates using bm25 ...

Finished Searching!
1 Legal Services Rendered Through January
2 GERS Studies-Pharmacy Sell In/Sell out D
3 Bmab1800 RLD proc at Syngene
------------------------------